In [1]:
# %% [code]
import os, json, time, warnings, math
from dataclasses import dataclass, asdict
from typing import Dict, Any, Tuple, List, Optional

import numpy as np
import pandas as pd

from joblib import Parallel, delayed
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LassoCV, LogisticRegression, HuberRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# ---- prevent nested threading (VERY IMPORTANT) ----
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

OUTDIR = "./gapA_outputs_v3"
os.makedirs(OUTDIR, exist_ok=True)
print("OUTDIR =", os.path.abspath(OUTDIR))


OUTDIR = c:\Users\starw\.vscode\practice\gapA_outputs_v3


In [2]:
# %% [code]
@dataclass
class Config:
    # ========== Main Monte Carlo ==========
    RUN_MAIN_MC: bool = True
    MAIN_REPS: int = 400          # 1000은 과하니 400으로 "안정성+현실성" 타협
    N: int = 1000
    P: int = 50
    THETA0: float = 0.5
    
    # DML
    K_FOLDS: int = 5
    SEED: int = 42
    
    # winsor levels for score-winsor sweep (cheap)
    WINSOR_LEVELS: Tuple[float, ...] = (0.0, 0.005, 0.01, 0.025, 0.05)

    # ========== Learners ==========
    RUN_LASSO: bool = True
    RUN_RF: bool = True
    RF_NEST: int = 300
    RF_MAXDEPTH: int = 8
    RUN_ROBUST_NUISANCE: bool = True   # Huber nuisance
    
    # ========== Tail / contamination knobs ==========
    STUDENT_T_DF_MAIN: int = 3
    STUDENT_T_DF_SWEEP: int = 10       # (추가3) tail severity sweep
    CONTAM_EPS: float = 0.05
    CONTAM_SCALE: float = 25.0
    
    # ========== Appendix (Expensive strategies) ==========
    RUN_APPX_EXPENSIVE: bool = True
    APPX_REPS: int = 200               # expensive는 reps를 줄여도 방어용으론 충분
    EXPENSIVE_WINSOR_LEVELS: Tuple[float, ...] = (0.01, 0.05)
    EXPENSIVE_DGPS: Tuple[str, ...] = ("contamination", "student_t_df3")  # 핵심 꼬리 환경 2개만
    EXPENSIVE_LEARNERS: Tuple[str, ...] = ("lasso_logit", "rf_rf")        # 대표 2개만
    
    # ========== Parallel ==========
    N_JOBS: int = -1
    BATCH_SIZE: int = 8
    
    # ========== Output ==========
    SAVE_TABLES: bool = True
    SAVE_FIGS: bool = True

CFG = Config()
CFG


Config(RUN_MAIN_MC=True, MAIN_REPS=400, N=1000, P=50, THETA0=0.5, K_FOLDS=5, SEED=42, WINSOR_LEVELS=(0.0, 0.005, 0.01, 0.025, 0.05), RUN_LASSO=True, RUN_RF=True, RF_NEST=300, RF_MAXDEPTH=8, RUN_ROBUST_NUISANCE=True, STUDENT_T_DF_MAIN=3, STUDENT_T_DF_SWEEP=10, CONTAM_EPS=0.05, CONTAM_SCALE=25.0, RUN_APPX_EXPENSIVE=True, APPX_REPS=200, EXPENSIVE_WINSOR_LEVELS=(0.01, 0.05), EXPENSIVE_DGPS=('contamination', 'student_t_df3'), EXPENSIVE_LEARNERS=('lasso_logit', 'rf_rf'), N_JOBS=-1, BATCH_SIZE=8, SAVE_TABLES=True, SAVE_FIGS=True)

In [3]:
# %% [code]
def winsorize_array(x: np.ndarray, level: float) -> np.ndarray:
    if level <= 0:
        return x.copy()
    lo = np.quantile(x, level)
    hi = np.quantile(x, 1 - level)
    return np.clip(x, lo, hi)

def winsorize_matrix_by_col(X: np.ndarray, level: float) -> np.ndarray:
    if level <= 0:
        return X.copy()
    Xw = X.copy()
    for j in range(Xw.shape[1]):
        Xw[:, j] = winsorize_array(Xw[:, j], level)
    return Xw

def foldwise_cutoffs(train_col: np.ndarray, level: float) -> Tuple[float, float]:
    lo = np.quantile(train_col, level)
    hi = np.quantile(train_col, 1 - level)
    return lo, hi

def apply_cutoff(x: np.ndarray, lo: float, hi: float) -> np.ndarray:
    return np.clip(x, lo, hi)


In [4]:
# %% [code]
def make_learners(cfg: Config) -> Dict[str, Tuple[Any, Any]]:
    learners = {}
    if cfg.RUN_LASSO:
        g_lasso = Pipeline([
            ("scaler", StandardScaler()),
            ("model", LassoCV(cv=5, random_state=cfg.SEED, n_alphas=50, max_iter=5000))
        ])
        m_logit = Pipeline([
            ("scaler", StandardScaler()),
            ("model", LogisticRegression(max_iter=2000, solver="lbfgs"))
        ])
        learners["lasso_logit"] = (g_lasso, m_logit)
    
    if cfg.RUN_RF:
        g_rf = RandomForestRegressor(
            n_estimators=cfg.RF_NEST,
            max_depth=cfg.RF_MAXDEPTH,
            random_state=cfg.SEED,
            n_jobs=1,                 # IMPORTANT
            min_samples_leaf=10
        )
        m_rf = RandomForestClassifier(
            n_estimators=cfg.RF_NEST,
            max_depth=cfg.RF_MAXDEPTH,
            random_state=cfg.SEED,
            n_jobs=1,                 # IMPORTANT
            min_samples_leaf=10
        )
        learners["rf_rf"] = (g_rf, m_rf)
    return learners

def make_huber_strategy(cfg: Config) -> Tuple[Any, Any]:
    g_huber = Pipeline([("scaler", StandardScaler()), ("model", HuberRegressor())])
    m_logit = Pipeline([("scaler", StandardScaler()), ("model", LogisticRegression(max_iter=2000, solver="lbfgs"))])
    return g_huber, m_logit

LEARNERS = make_learners(CFG)
LEARNERS.keys()


dict_keys(['lasso_logit', 'rf_rf'])

In [5]:
# %% [code]
def crossfit_nuisance(
    X: np.ndarray, D: np.ndarray, Y: np.ndarray,
    g_learner, m_learner,
    k_folds: int, seed: int
) -> Tuple[np.ndarray, np.ndarray]:
    n = X.shape[0]
    ghat = np.zeros(n)
    mhat = np.zeros(n)
    kf = KFold(n_splits=k_folds, shuffle=True, random_state=seed)
    
    for tr_idx, te_idx in kf.split(X):
        X_tr, X_te = X[tr_idx], X[te_idx]
        Y_tr, D_tr = Y[tr_idx], D[tr_idx]
        
        g = g_learner
        m = m_learner
        
        g.fit(X_tr, Y_tr)
        ghat[te_idx] = g.predict(X_te)
        
        if hasattr(m, "predict_proba"):
            m.fit(X_tr, D_tr)
            mhat[te_idx] = m.predict_proba(X_te)[:, 1]
        else:
            m.fit(X_tr, D_tr)
            mhat[te_idx] = m.predict(X_te)
    
    return ghat, mhat

def dml_theta_se_from_residuals(D_res: np.ndarray, Y_res: np.ndarray) -> Dict[str, float]:
    n = len(D_res)
    denom = np.mean(D_res**2)
    theta_hat = np.mean(D_res * Y_res) / denom
    
    psi = (D_res * (Y_res - theta_hat * D_res)) / denom
    se = np.sqrt(np.var(psi, ddof=1) / n)
    
    return {
        "theta_hat": float(theta_hat),
        "se": float(se),
        "ci_lo": float(theta_hat - 1.96 * se),
        "ci_hi": float(theta_hat + 1.96 * se),
        "denom_E_Dres2": float(denom),
    }

def dml_plr_cached(D: np.ndarray, Y: np.ndarray, ghat: np.ndarray, mhat: np.ndarray, score_winsor: float) -> Dict[str, float]:
    D_res = D - mhat
    Y_res = Y - ghat
    if score_winsor > 0:
        D_res = winsorize_array(D_res, score_winsor)
        Y_res = winsorize_array(Y_res, score_winsor)
    return dml_theta_se_from_residuals(D_res, Y_res)


In [6]:
# %% [code]
def gen_dgp(cfg: Config, dgp_name: str, rng: np.random.Generator) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    dgp_name options:
      - gaussian
      - student_t_df3
      - student_t_df10   (tail sweep)
      - hetero_t_df3
      - contamination
    """
    N, P = cfg.N, cfg.P
    
    X = rng.normal(size=(N, P))
    fX = (X[:, 0] + 0.5*X[:, 1]**2 - 0.25*np.sin(X[:, 2]))
    
    linp = 0.6*X[:, 0] - 0.4*X[:, 1] + 0.2*X[:, 2]
    pscore = 1 / (1 + np.exp(-1.0 * linp))
    D = rng.binomial(1, pscore, size=N).astype(float)
    
    if dgp_name == "gaussian":
        U = rng.normal(size=N)
    elif dgp_name == "student_t_df3":
        U = rng.standard_t(df=cfg.STUDENT_T_DF_MAIN, size=N)
    elif dgp_name == "student_t_df10":
        U = rng.standard_t(df=cfg.STUDENT_T_DF_SWEEP, size=N)
    elif dgp_name == "hetero_t_df3":
        scale = np.exp(0.3 * X[:, 0])
        U = scale * rng.standard_t(df=cfg.STUDENT_T_DF_MAIN, size=N)
    elif dgp_name == "contamination":
        base = rng.normal(size=N)
        shock = rng.normal(scale=cfg.CONTAM_SCALE, size=N)
        mask = rng.uniform(size=N) < cfg.CONTAM_EPS
        U = np.where(mask, shock, base)
    else:
        raise ValueError("Unknown dgp_name")
    
    Y = cfg.THETA0 * D + fX + U
    return X, D, Y

DGP_LIST_MAIN = ("gaussian", "student_t_df3", "student_t_df10", "hetero_t_df3", "contamination")
DGP_LIST_MAIN


('gaussian',
 'student_t_df3',
 'student_t_df10',
 'hetero_t_df3',
 'contamination')

In [7]:
# %% [code]
def run_one_rep_dgp_main(cfg: Config, rep: int, dgp: str) -> List[Dict[str, Any]]:
    base_seed = cfg.SEED * 100000 + rep * 10 + (abs(hash(dgp)) % 10)
    rng = np.random.default_rng(base_seed)
    X, D, Y = gen_dgp(cfg, dgp, rng)
    
    rows: List[Dict[str, Any]] = []
    learners = make_learners(cfg)
    
    for learner_key, (g, m) in learners.items():
        ghat, mhat = crossfit_nuisance(X, D, Y, g, m, cfg.K_FOLDS, cfg.SEED)
        
        # baseline
        r0 = dml_plr_cached(D, Y, ghat, mhat, score_winsor=0.0)
        rows.append({"rep": rep, "dgp": dgp, "learner": learner_key, "strategy": "S0_baseline", "winsor_level": 0.0, **r0})
        
        # score-winsor sweep (cheap)
        for w in cfg.WINSOR_LEVELS:
            if w <= 0: 
                continue
            r2 = dml_plr_cached(D, Y, ghat, mhat, score_winsor=float(w))
            rows.append({"rep": rep, "dgp": dgp, "learner": learner_key, "strategy": "S2_score_winsor", "winsor_level": float(w), **r2})
    
    # robust nuisance once per rep/dgp
    if cfg.RUN_ROBUST_NUISANCE:
        g_h, m_l = make_huber_strategy(cfg)
        ghat_h, mhat_h = crossfit_nuisance(X, D, Y, g_h, m_l, cfg.K_FOLDS, cfg.SEED)
        r3 = dml_plr_cached(D, Y, ghat_h, mhat_h, score_winsor=0.0)
        rows.append({"rep": rep, "dgp": dgp, "learner": "huber_logit", "strategy": "S3_robust_nuisance", "winsor_level": 0.0, **r3})
    
    return rows


In [8]:
# %% [code]
def summarize_mc_with_cov_ci(df: pd.DataFrame, theta0: float) -> pd.DataFrame:
    g = df.groupby(["dgp", "learner", "strategy", "winsor_level"], as_index=False)
    
    out = g.agg(
        mean_theta=("theta_hat", "mean"),
        bias=("theta_hat", lambda x: float(np.mean(x - theta0))),
        rmse=("theta_hat", lambda x: float(np.sqrt(np.mean((x - theta0)**2)))),
        mean_se=("se", "mean"),
        mean_width=("ci_hi", lambda hi: float(np.mean(hi - df.loc[hi.index, "ci_lo"]))),
        n_reps=("theta_hat", "count"),
        cov_rate=("theta_hat", lambda x: float(np.mean(
            (df.loc[x.index, "ci_lo"] <= theta0) & (theta0 <= df.loc[x.index, "ci_hi"])
        )))
    )
    # Binomial SE and CI for coverage
    p = out["cov_rate"].to_numpy()
    n = out["n_reps"].to_numpy()
    cov_se = np.sqrt(np.maximum(p * (1 - p) / n, 0))
    out["cov_se"] = cov_se
    out["cov_ci_lo"] = np.clip(p - 1.96 * cov_se, 0, 1)
    out["cov_ci_hi"] = np.clip(p + 1.96 * cov_se, 0, 1)
    out = out.rename(columns={"cov_rate": "coverage"})
    return out

RAW_MAIN, SUM_MAIN = None, None

if CFG.RUN_MAIN_MC:
    t0 = time.time()
    tasks = [(rep, dgp) for rep in range(CFG.MAIN_REPS) for dgp in DGP_LIST_MAIN]
    print(f"MAIN tasks = {len(tasks)} = reps({CFG.MAIN_REPS}) × dgp({len(DGP_LIST_MAIN)})")
    
    blocks = Parallel(n_jobs=CFG.N_JOBS, verbose=10, batch_size=CFG.BATCH_SIZE)(
        delayed(run_one_rep_dgp_main)(CFG, rep, dgp) for (rep, dgp) in tasks
    )
    flat = [r for blk in blocks for r in blk]
    RAW_MAIN = pd.DataFrame(flat)
    SUM_MAIN = summarize_mc_with_cov_ci(RAW_MAIN, CFG.THETA0)
    
    print("MAIN MC finished in sec:", round(time.time() - t0, 1))
    print(SUM_MAIN.sort_values(["dgp","learner","strategy","winsor_level"]).head(12))
    
    if CFG.SAVE_TABLES:
        RAW_MAIN.to_csv(os.path.join(OUTDIR, "main_mc_raw.csv"), index=False)
        SUM_MAIN.to_csv(os.path.join(OUTDIR, "main_mc_summary.csv"), index=False)
        print("Saved:", os.path.abspath(os.path.join(OUTDIR, "main_mc_raw.csv")))
        print("Saved:", os.path.abspath(os.path.join(OUTDIR, "main_mc_summary.csv")))


MAIN tasks = 2000 = reps(400) × dgp(5)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed: 11.6min
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed: 21.3min
[Parallel(n_jobs=-1)]: Done 232 tasks      | elapsed: 31.1min
[Parallel(n_jobs=-1)]: Done 320 tasks      | elapsed: 40.6min
[Parallel(n_jobs=-1)]: Done 408 tasks      | elapsed: 42.1min
[Parallel(n_jobs=-1)]: Done 512 tasks      | elapsed: 61.7min
[Parallel(n_jobs=-1)]: Done 616 tasks      | elapsed: 73.9min
[Parallel(n_jobs=-1)]: Done 736 tasks      | elapsed: 86.2min
[Parallel(n_jobs=-1)]: Done 856 tasks      | elapsed: 154.4min
[Parallel(n_jobs=-1)]: Done 992 tasks      | elapsed: 175.2min
[Parallel(n_jobs=-1)]: Done 1128 tasks      | elapsed: 186.7min
[Parallel(n_jobs=-1)]: Done 1280 tasks      | elaps

MAIN MC finished in sec: 17660.1
              dgp      learner            strategy  winsor_level  mean_theta  \
0   contamination  huber_logit  S3_robust_nuisance         0.000    0.489860   
1   contamination  lasso_logit         S0_baseline         0.000    0.461554   
2   contamination  lasso_logit     S2_score_winsor         0.005    0.463072   
3   contamination  lasso_logit     S2_score_winsor         0.010    0.459847   
4   contamination  lasso_logit     S2_score_winsor         0.025    0.457467   
5   contamination  lasso_logit     S2_score_winsor         0.050    0.450329   
6   contamination        rf_rf         S0_baseline         0.000    0.451139   
7   contamination        rf_rf     S2_score_winsor         0.005    0.452198   
8   contamination        rf_rf     S2_score_winsor         0.010    0.446843   
9   contamination        rf_rf     S2_score_winsor         0.025    0.436875   
10  contamination        rf_rf     S2_score_winsor         0.050    0.418586   
11     

In [9]:
# %% [code]
def dml_plr_honest_foldwise(
    X: np.ndarray, D: np.ndarray, Y: np.ndarray,
    g_learner, m_learner,
    k_folds: int, seed: int,
    winsor_level: float
) -> Dict[str, float]:
    n, p = X.shape
    ghat = np.zeros(n)
    mhat = np.zeros(n)
    kf = KFold(n_splits=k_folds, shuffle=True, random_state=seed)
    
    for tr_idx, te_idx in kf.split(X):
        X_tr, X_te = X[tr_idx].copy(), X[te_idx].copy()
        Y_tr, D_tr = Y[tr_idx].copy(), D[tr_idx].copy()
        
        if winsor_level > 0:
            # X cutoffs from TRAIN only
            for j in range(p):
                lo, hi = foldwise_cutoffs(X_tr[:, j], winsor_level)
                X_tr[:, j] = apply_cutoff(X_tr[:, j], lo, hi)
                X_te[:, j] = apply_cutoff(X_te[:, j], lo, hi)
            # Y cutoffs from TRAIN only
            loY, hiY = foldwise_cutoffs(Y_tr, winsor_level)
            Y_tr = apply_cutoff(Y_tr, loY, hiY)
        
        g = g_learner
        m = m_learner
        
        g.fit(X_tr, Y_tr)
        ghat[te_idx] = g.predict(X_te)
        
        if hasattr(m, "predict_proba"):
            m.fit(X_tr, D_tr)
            mhat[te_idx] = m.predict_proba(X_te)[:, 1]
        else:
            m.fit(X_tr, D_tr)
            mhat[te_idx] = m.predict(X_te)
    
    return dml_plr_cached(D, Y, ghat, mhat, score_winsor=0.0)

def run_one_rep_dgp_expensive(cfg: Config, rep: int, dgp: str) -> List[Dict[str, Any]]:
    base_seed = cfg.SEED * 200000 + rep * 10 + (abs(hash(dgp)) % 10)
    rng = np.random.default_rng(base_seed)
    X, D, Y = gen_dgp(cfg, dgp, rng)
    
    rows: List[Dict[str, Any]] = []
    learners = make_learners(cfg)
    
    for learner_key, (g, m) in learners.items():
        if learner_key not in cfg.EXPENSIVE_LEARNERS:
            continue
        
        for w in cfg.EXPENSIVE_WINSOR_LEVELS:
            # S1: data-winsor (refit nuisance)
            Xw = winsorize_matrix_by_col(X, w)
            Yw = winsorize_array(Y, w)
            ghat_w, mhat_w = crossfit_nuisance(Xw, D, Yw, g, m, cfg.K_FOLDS, cfg.SEED)
            r1 = dml_plr_cached(D, Yw, ghat_w, mhat_w, score_winsor=0.0)
            rows.append({"rep": rep, "dgp": dgp, "learner": learner_key, "strategy": "S1_data_winsor", "winsor_level": float(w), **r1})
            
            # S4: honest winsor (refit nuisance within fold with train cutoffs)
            r4 = dml_plr_honest_foldwise(X, D, Y, g, m, cfg.K_FOLDS, cfg.SEED, float(w))
            rows.append({"rep": rep, "dgp": dgp, "learner": learner_key, "strategy": "S4_honest_winsor", "winsor_level": float(w), **r4})
    
    return rows

RAW_APPX, SUM_APPX = None, None

if CFG.RUN_APPX_EXPENSIVE:
    t0 = time.time()
    tasks = [(rep, dgp) for rep in range(CFG.APPX_REPS) for dgp in CFG.EXPENSIVE_DGPS]
    print(f"APPX(expensive) tasks = {len(tasks)} = reps({CFG.APPX_REPS}) × dgp({len(CFG.EXPENSIVE_DGPS)})")
    
    blocks = Parallel(n_jobs=CFG.N_JOBS, verbose=10, batch_size=max(1, CFG.BATCH_SIZE))(
        delayed(run_one_rep_dgp_expensive)(CFG, rep, dgp) for (rep, dgp) in tasks
    )
    flat = [r for blk in blocks for r in blk]
    RAW_APPX = pd.DataFrame(flat)
    SUM_APPX = summarize_mc_with_cov_ci(RAW_APPX, CFG.THETA0)
    
    print("APPX finished in sec:", round(time.time() - t0, 1))
    print(SUM_APPX.sort_values(["dgp","learner","strategy","winsor_level"]).head(12))
    
    if CFG.SAVE_TABLES:
        RAW_APPX.to_csv(os.path.join(OUTDIR, "appx_expensive_raw.csv"), index=False)
        SUM_APPX.to_csv(os.path.join(OUTDIR, "appx_expensive_summary.csv"), index=False)
        print("Saved:", os.path.abspath(os.path.join(OUTDIR, "appx_expensive_raw.csv")))
        print("Saved:", os.path.abspath(os.path.join(OUTDIR, "appx_expensive_summary.csv")))


APPX(expensive) tasks = 400 = reps(200) × dgp(2)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 13.5min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed: 13.7min
[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed: 72.4min
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed: 73.5min
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed: 117.1min
[Parallel(n_jobs=-1)]: Done 232 tasks      | elapsed: 156.7min
[Parallel(n_jobs=-1)]: Done 313 tasks      | elapsed: 160.3min
[Parallel(n_jobs=-1)]: Done 324 tasks      | elapsed: 163.9min
[Parallel(n_jobs=-1)]: Done 337 tasks      | elapsed: 168.8min
[Parallel(n_jobs=-1)]: Done 350 tasks      | elapsed: 173.2min
[Parallel(n_jobs=-1)]: Done 365 tasks      | elapsed: 178.4min
[Parallel(n_jobs=-1)]: Done 380 tasks      | elapsed: 183.3min


APPX finished in sec: 12359.3
              dgp      learner          strategy  winsor_level  mean_theta  \
0   contamination  lasso_logit    S1_data_winsor          0.01    0.468732   
1   contamination  lasso_logit    S1_data_winsor          0.05    0.434932   
2   contamination  lasso_logit  S4_honest_winsor          0.01    0.486729   
3   contamination  lasso_logit  S4_honest_winsor          0.05    0.496800   
4   contamination        rf_rf    S1_data_winsor          0.01    0.461549   
5   contamination        rf_rf    S1_data_winsor          0.05    0.441490   
6   contamination        rf_rf  S4_honest_winsor          0.01    0.482576   
7   contamination        rf_rf  S4_honest_winsor          0.05    0.532026   
8   student_t_df3  lasso_logit    S1_data_winsor          0.01    0.459919   
9   student_t_df3  lasso_logit    S1_data_winsor          0.05    0.435490   
10  student_t_df3  lasso_logit  S4_honest_winsor          0.01    0.468579   
11  student_t_df3  lasso_logit  S4

[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed: 206.0min finished


In [10]:
# %% [code]
def plot_metric_vs_winsor(sum_df: pd.DataFrame, metric: str, out_prefix: str):
    """
    metric: "coverage", "mean_width", "rmse", "bias"
    """
    for dgp in sorted(sum_df["dgp"].unique()):
        sub = sum_df[sum_df["dgp"] == dgp].copy()
        for learner in sorted(sub["learner"].unique()):
            sub2 = sub[sub["learner"] == learner].copy()
            
            plt.figure(figsize=(9,5))
            for strat in sorted(sub2["strategy"].unique()):
                s3 = sub2[sub2["strategy"] == strat].sort_values("winsor_level")
                plt.plot(s3["winsor_level"], s3[metric], marker="o", label=strat)
            if metric == "coverage":
                plt.axhline(0.95, linestyle="--")
            plt.title(f"{metric} vs winsor_level | dgp={dgp} | learner={learner}")
            plt.xlabel("winsor_level")
            plt.ylabel(metric)
            plt.legend()
            plt.tight_layout()
            
            if CFG.SAVE_FIGS:
                path = os.path.join(OUTDIR, f"{out_prefix}_{metric}_{dgp}_{learner}.png")
                plt.savefig(path, dpi=200)
                plt.close()
            else:
                plt.show()

if (SUM_MAIN is not None) and CFG.SAVE_FIGS:
    plot_metric_vs_winsor(SUM_MAIN, "coverage", "fig_main")
    plot_metric_vs_winsor(SUM_MAIN, "mean_width", "fig_main")
    plot_metric_vs_winsor(SUM_MAIN, "rmse", "fig_main")
    print("Saved figures under:", os.path.abspath(OUTDIR))


Saved figures under: c:\Users\starw\.vscode\practice\gapA_outputs_v3


In [11]:
# %% [code]
bundle = {
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "config": asdict(CFG),
    "outdir": os.path.abspath(OUTDIR),
    "files": {
        "main_mc_raw": os.path.abspath(os.path.join(OUTDIR, "main_mc_raw.csv")),
        "main_mc_summary": os.path.abspath(os.path.join(OUTDIR, "main_mc_summary.csv")),
        "appx_expensive_raw": os.path.abspath(os.path.join(OUTDIR, "appx_expensive_raw.csv")),
        "appx_expensive_summary": os.path.abspath(os.path.join(OUTDIR, "appx_expensive_summary.csv")),
    },
    "main_head": None,
    "appx_head": None
}

if SUM_MAIN is not None:
    bundle["main_head"] = SUM_MAIN.sort_values(["dgp","learner","strategy","winsor_level"]).head(40).to_dict(orient="records")
if SUM_APPX is not None:
    bundle["appx_head"] = SUM_APPX.sort_values(["dgp","learner","strategy","winsor_level"]).head(40).to_dict(orient="records")

print(json.dumps(bundle, ensure_ascii=False, indent=2)[:20000])


{
  "timestamp": "2026-01-06 19:29:25",
  "config": {
    "RUN_MAIN_MC": true,
    "MAIN_REPS": 400,
    "N": 1000,
    "P": 50,
    "THETA0": 0.5,
    "K_FOLDS": 5,
    "SEED": 42,
    "WINSOR_LEVELS": [
      0.0,
      0.005,
      0.01,
      0.025,
      0.05
    ],
    "RUN_LASSO": true,
    "RUN_RF": true,
    "RF_NEST": 300,
    "RF_MAXDEPTH": 8,
    "RUN_ROBUST_NUISANCE": true,
    "STUDENT_T_DF_MAIN": 3,
    "STUDENT_T_DF_SWEEP": 10,
    "CONTAM_EPS": 0.05,
    "CONTAM_SCALE": 25.0,
    "RUN_APPX_EXPENSIVE": true,
    "APPX_REPS": 200,
    "EXPENSIVE_WINSOR_LEVELS": [
      0.01,
      0.05
    ],
    "EXPENSIVE_DGPS": [
      "contamination",
      "student_t_df3"
    ],
    "EXPENSIVE_LEARNERS": [
      "lasso_logit",
      "rf_rf"
    ],
    "N_JOBS": -1,
    "BATCH_SIZE": 8,
    "SAVE_TABLES": true,
    "SAVE_FIGS": true
  },
  "outdir": "c:\\Users\\starw\\.vscode\\practice\\gapA_outputs_v3",
  "files": {
    "main_mc_raw": "c:\\Users\\starw\\.vscode\\practice\\gapA_outpu